In [161]:
import pandas as pd
import seaborn as sns
df = pd.read_csv(r'C:\work\Data_analyst\Test\Космос\TZ_GP.csv')

In [162]:
# Зададим настройки графиков
sns.set(
    font_scale=2,
    style="whitegrid",
    rc={'figure.figsize':(20,7)}
        )


In [152]:
df

,user_id,media_source,client_time,session_index,game_status,event_name,event_data
0,60c588d7b4d1b24b40531f29,organic,2021-06-05 07:23:06,1,"{""coins"":0,""freeRounds"":0,""freeUndo"":0,""freeWi...",appInstall,"{""device"":""Xiaomi POCO M2 Pro"",""graphicMemory""..."
1,60d34c7ab4d1b24b405324ac,organic,2021-06-05 16:33:26,1,"{""coins"":0,""freeRounds"":0,""freeUndo"":0,""freeWi...",appInstall,"{""device"":""samsung SM-J200G"",""graphicMemory"":2..."
2,60babef87c80da6a9442cb86,ad-traffic,2021-06-05 00:02:07,1,"{""coins"":0,""freeRounds"":0,""freeUndo"":0,""freeWi...",appInstall,"{""device"":""samsung SM-G781B"",""graphicMemory"":2..."
3,60bac0e912667e6a8d107731,ad-traffic,2021-06-05 00:04:29,1,"{""coins"":0,""freeRounds"":0,""freeUndo"":0,""freeWi...",appInstall,"{""device"":""samsung SM-T290"",""graphicMemory"":51..."
4,60babfb17c80da6a9442cb87,organic,2021-06-05 00:05:16,1,"{""coins"":0,""freeRounds"":0,""freeUndo"":0,""freeWi...",appInstall,"{""device"":""samsung SM-N960F"",""graphicMemory"":2..."
...,...,...,...,...,...,...,...
396094,60bb2f3912667e6a8d107757,organic,2021-06-12 11:52:41,54,"{""coins"":1174190,""freeRounds"":30,""freeUndo"":30...",transactionComplete,"{""inLevel"":false,""usdProceed"":68.1768094378241..."
396095,60bb2f3912667e6a8d107757,organic,2021-06-12 15:18:06,57,"{""coins"":7885,""freeRounds"":0,""freeUndo"":394,""f...",transactionComplete,"{""inLevel"":false,""usdProceed"":6.81001009386441..."
396096,60bb2f3912667e6a8d107757,organic,2021-06-12 15:18:30,57,"{""coins"":107885,""freeRounds"":0,""freeUndo"":394,...",transactionComplete,"{""inLevel"":false,""usdProceed"":13.6285433543043..."
396097,60bc090c12667e6a8d1077b8,organic,2021-06-12 22:48:26,2,"{""coins"":45274,""freeRounds"":0,""freeUndo"":3,""fr...",transactionComplete,"{""inLevel"":false,""usdProceed"":1.393,""productDe..."


In [163]:
df['client_time'] = pd.to_datetime(df.client_time)

In [137]:
# Посчитаем количество установок
df.query('event_name =="appInstall"').user_id.nunique()

452

In [139]:
# Посчитаем количество обучений
df.query('event_name =="learning"').user_id.nunique()

446

In [140]:
# На каком уровне проходят первую грядку?
# 292 игрока смогли пройти первую грядку
# 25% игорков - на 9 уровне
# 50 % всех игроков на 10
# 75% игроков - на 11
level_first_glade = df.query('event_name =="gladeComplete"')\
    .sort_values('client_time') \
    .groupby('user_id', as_index = False)\
    .head(1) \
    .event_data.str.extract(r'"levels":(\d*)')
level_first_glade.columns = ['level']
level_first_glade= level_first_glade.astype({'level':'int32'})
level_first_glade.describe()

,level
count,292.000000
mean,14.547945
std,36.812720
min,1.000000
25%,9.000000
50%,10.000000
75%,11.000000
max,464.000000


In [141]:
# На каком уровне проходят первую грядку в абсолютных значениях
level_first_glade.value_counts()

level
10       103
9         73
11        39
8         33
12        18
7          5
13         5
1          2
6          2
15         2
17         2
151        1
356        1
168        1
14         1
127        1
105        1
89         1
464        1
dtype: int64

In [142]:
# Сколько сессий за время наблюдений сделали игроки?
# Медианное значение числа сессий (50% игроков) - 5 
df.session_index.describe()

count    396099.000000
mean         15.308903
std          27.443017
min           1.000000
25%           2.000000
50%           5.000000
75%          16.000000
max         289.000000
Name: session_index, dtype: float64

In [196]:
# Посмотрим на получение звезд в зависимости от уровня 
# Уровни 0-5
stars_level_0_5 = df\
    .query('session_index <= 5 & event_name == "getStarsCruise"') \
    .groupby('user_id') \
    .event_data \
    .value_counts()
stars_level_0_5 = stars_level_0_5.to_frame().rename(columns={'event_data': 'count'}).reset_index()
stars_level_0_5['stars'] = stars_level_0_5.event_data.str.extract(r'"amount":(\d*)}')
stars_level_0_5 = stars_level_0_5.astype({'stars':'int32'})
stars_level_0_5['stars_total_0_5'] = stars_level_0_5['count']*stars_level_0_5['stars']
# 50% игроков получили за первые 5 сессий 100 звезд
stars_level_0_5.groupby('user_id').agg({'stars_total_0_5':sum}).describe()

,stars_total_0_5
count,335.000000
mean,116.901493
std,94.817817
min,5.000000
25%,41.000000
50%,100.000000
75%,176.500000
max,519.000000


In [197]:
# Уровни 5-10
stars_level_5_10 = df\
    .query('session_index > 5 & session_index <= 10 & event_name == "getStarsCruise"') \
    .groupby('user_id') \
    .event_data \
    .value_counts()
stars_level_5_10 = stars_level_5_10.to_frame().rename(columns={'event_data': 'count'}).reset_index()
stars_level_5_10['stars'] = stars_level_5_10.event_data.str.extract(r'"amount":(\d*)}')
stars_level_5_10 = stars_level_5_10.astype({'stars':'int32'})
stars_level_5_10['stars_total_5_10'] = stars_level_5_10['count']*stars_level_5_10['stars']
# 50% игроков получили за сессии с 5 по 10 - 34 звезд. Меньше, потому что играют меньше или уровни становятся сложнее?
stars_level_5_10.groupby('user_id').agg({'stars_total_5_10':sum}).describe()

,stars_total_5_10
count,181.000000
mean,50.773481
std,60.765200
min,2.000000
25%,14.000000
50%,34.000000
75%,68.000000
max,466.000000


In [132]:
# Получение бустеров по дням: в первый день больше всего (больше установок - больше играют),
# небольшое увеличение 2021-06-12 - их купили или выходной? 
df.query('event_name =="getBooster"')\
    .resample(rule='D')\
    .event_data \
    .count()

client_time
2021-06-05    14347
2021-06-06     3446
2021-06-07     1739
2021-06-08     1956
2021-06-09     1983
2021-06-10     1397
2021-06-11     1718
2021-06-12     2581
Freq: D, Name: event_data, dtype: int64